In [ ]:
import xarray as xr
import xclim.indices as xci
import numpy as np


In [ ]:
def process_precipitation_data(input_files, output_file):
    # 读取并合并多个 NetCDF 文件
    datasets = [xr.open_dataset(file, engine='netcdf4') for file in input_files]
    ds = xr.concat(datasets, dim='time')
    
    # 确保时间维度转换正确
    if 'time' in ds.coords:
        ds['time'] = xr.cftime_range(start='1961-01-01', periods=ds.sizes['time'], freq='D')
    
    # 获取降水数据变量
    pr = ds['pre']
    
    # 确保数据类型适合计算，转换为 float32
    pr = pr.astype(np.float32)
    
    # 为降水数据添加单位属性
    pr.attrs['units'] = 'mm/day'
    
    # 将日降水量小于0和大于700的点设置为 NaN
    pr = pr.where((pr >= 0) & (pr <= 700), np.nan)
    
    # 计算各种降水指数
    wetdays = xci.wetdays(pr, thresh='1.0 mm/day', freq='YS', op='>=')
    wetdays_prop = xci.wetdays_prop(pr, thresh='1.0 mm/day', freq='YS', op='>=')
    prcptot = xci.prcptot(pr, thresh='1 mm/day', freq='YS')
    daily_pr_intensity = xci.daily_pr_intensity(pr, thresh='1 mm/day', freq='YS', op='>=')
    max_consec_dry_days = xci.maximum_consecutive_dry_days(pr, thresh='1 mm/day', freq='YS', resample_before_rl=True)
    max_consec_wet_days = xci.maximum_consecutive_wet_days(pr, thresh='1 mm/day', freq='YS', resample_before_rl=True)
    max_1_day_precip = xci.max_n_day_precipitation_amount(pr, window=1, freq='YS')
    max_5_day_precip = xci.max_n_day_precipitation_amount(pr, window=5, freq='YS')
    wetdays_10mm = xci.wetdays(pr, thresh='10.0 mm/day', freq='YS', op='>=')
    wetdays_20mm = xci.wetdays(pr, thresh='20.0 mm/day', freq='YS', op='>=')
    
    # 创建一个新的 Dataset 保存结果
    result_ds = xr.Dataset({
        'wetdays': wetdays,
        'wetdays_prop': wetdays_prop,
        'prcptot': prcptot,
        'daily_pr_intensity': daily_pr_intensity,
        'max_consec_dry_days': max_consec_dry_days,
        'max_consec_wet_days': max_consec_wet_days,
        'max_1_day_precip': max_1_day_precip,
        'max_5_day_precip': max_5_day_precip,
        'wetdays_10mm': wetdays_10mm,
        'wetdays_20mm': wetdays_20mm
    })
    
    # 添加属性
    result_ds['wetdays'].attrs = {
        'long_name': 'Wet Days',
        'units': 'days',
        'description': 'Number of wet days per year'
    }
    result_ds['wetdays_prop'].attrs = {
        'long_name': 'Wet Days Proportion',
        'units': 'fraction',
        'description': 'Proportion of wet days per year'
    }
    result_ds['prcptot'].attrs = {
        'long_name': 'Total Precipitation',
        'units': 'mm',
        'description': 'Total precipitation in wet days'
    }
    result_ds['daily_pr_intensity'].attrs = {
        'long_name': 'Daily Precipitation Intensity',
        'units': 'mm/day',
        'description': 'Mean daily precipitation intensity on wet days'
    }
    result_ds['max_consec_dry_days'].attrs = {
        'long_name': 'Maximum Consecutive Dry Days',
        'units': 'days',
        'description': 'Maximum number of consecutive dry days'
    }
    result_ds['max_consec_wet_days'].attrs = {
        'long_name': 'Maximum Consecutive Wet Days',
        'units': 'days',
        'description': 'Maximum number of consecutive wet days'
    }
    result_ds['max_1_day_precip'].attrs = {
        'long_name': 'Maximum 1-Day Precipitation',
        'units': 'mm',
        'description': 'Maximum 1-day precipitation amount'
    }
    result_ds['max_5_day_precip'].attrs = {
        'long_name': 'Maximum 5-Day Precipitation',
        'units': 'mm',
        'description': 'Maximum 5-day precipitation amount'
    }
    result_ds['wetdays_10mm'].attrs = {
        'long_name': 'Wet Days (>= 10 mm)',
        'units': 'days',
        'description': 'Number of wet days with precipitation >= 10 mm'
    }
    result_ds['wetdays_20mm'].attrs = {
        'long_name': 'Wet Days (>= 20 mm)',
        'units': 'days',
        'description': 'Number of wet days with precipitation >= 20 mm'
    }
    
    # 保存结果到新的 NetCDF 文件
    result_ds.to_netcdf(output_file, engine='netcdf4')  # 明确指定使用 'netcdf4' 引擎
    print(f"Saved precipitation indices to {output_file}")


In [ ]:
# 定义输入文件和输出文件路径
input_files = [
    r'D:\CMIP6extreme\00 - CN051-2022\1961-2021\CN05.1_Pre_1961_2021_daily_025x025.nc',
    r'D:\CMIP6extreme\00 - CN051-2022\2022年\CN05.1_Pre_2022_daily_025x025.nc'
]
output_file = r'D:\CMIP6extreme\00 - CN051-2022\extreme_pr\extreme_pr_index_1961_2022.nc'

# 调用函数进行计算并保存结果
process_precipitation_data(input_files, output_file)
